In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Bidirectional, GRU, Reshape
from tensorflow.keras.metrics import Precision, Recall
import matplotlib.pyplot as plt

In [2]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert probabilities to binary labels (threshold at 0.5)
        y_pred = tf.cast(y_pred >= 0.5, tf.float32)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [3]:
# Self-Attention Layer
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W_query = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)
        self.W_key = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)
        self.W_value = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)
        self.attention = tf.keras.layers.Attention()

    def call(self, inputs):
        query = tf.matmul(inputs, self.W_query)
        key = tf.matmul(inputs, self.W_key)
        value = tf.matmul(inputs, self.W_value)
        attention_output = self.attention([query, key, value])
        return attention_output

In [4]:
# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Set paths for your dataset
train_directory = r'Monkeypox/Kaggle MP images/archive (2)/Augmented Images/Augmented Images/'  # Update with the correct path for training images
test_directory = r'Monkeypox/Kaggle MP images/archive (2)/Augmented Images/Augmented Images/'  # Update with the correct path for testing images

In [6]:
# Image data generators for training and testing
training_set = train_datagen.flow_from_directory(
    train_directory,
    target_size=(64, 64),  # Change target size to 64x64
    batch_size=32,
    class_mode='binary'  # Change based on your dataset (e.g., 'binary', 'categorical', 'sparse')
)

test_set = test_datagen.flow_from_directory(
    test_directory,
    target_size=(64, 64),  # Change target size to 64x64
    batch_size=32,
    class_mode='binary'  # Change based on your dataset (e.g., 'binary', 'categorical', 'sparse')
)

Found 3192 images belonging to 2 classes.
Found 3192 images belonging to 2 classes.


In [7]:
# Model Definition with CNN + Self-Attention + Bidirectional GRU
model = Sequential()


In [8]:
# Add a Conv2D layer to process images
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))  # Input shape changed to (64, 64, 3)
model.add(MaxPooling2D(pool_size=(2, 2)))

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Add more Conv2D and MaxPooling layers as needed
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [10]:
# Reshape to make it compatible with GRU layer
model.add(Reshape((-1, 64)))  # Reshape to (batch_size, timesteps, features) for GRU

In [11]:
# Add Self-Attention Layer
model.add(SelfAttention())

In [12]:
# Add Bidirectional GRU layers
model.add(Bidirectional(GRU(64, activation='relu', return_sequences=False)))

In [13]:
# Add Dropout and Dense layers for classification
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  

In [14]:
# Compile the model with custom F1 score and other metrics
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  
    metrics=['accuracy', Precision(), Recall(), F1Score()]
)

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 196, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ self_attention (SelfAttention)       │ (None, 196, 64)             │          12,288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │          49,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 81,729 (319.25 KB)

 Trainable params: 81,729 (319.25 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=100,  # Adjust as needed
)

Epoch 1/100


C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 127ms/step - accuracy: 0.5718 - f1_score: 0.7033 - loss: 0.6784 - precision: 0.5763 - recall: 0.9122 - val_accuracy: 0.6143 - val_f1_score: 0.7343 - val_loss: 0.6517 - val_precision: 0.5929 - val_recall: 0.9643
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 150ms/step - accuracy: 0.6427 - f1_score: 0.7260 - loss: 0.6451 - precision: 0.6352 - recall: 0.8490 - val_accuracy: 0.6472 - val_f1_score: 0.6734 - val_loss: 0.6325 - val_precision: 0.6894 - val_recall: 0.6582
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 158ms/step - accuracy: 0.6802 - f1_score: 0.7428 - loss: 0.6219 - precision: 0.6751 - recall: 0.8285 - val_accuracy: 0.6591 - val_f1_score: 0.7478 - val_loss: 0.6298 - val_precision: 0.6325 - val_recall: 0.9144
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 154ms/step - accuracy: 0.6665 - f1_score: 0.7385 - loss: 0.6255 - precision: 0.6536 - recall: 0.8500 - val_accuracy: 0.6404 - val_f1_score: 0.7447 - val_loss: 0.6409 - val_precision: 0.6127 - val_rec

In [17]:
# Save Loss plot
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('loss_plot_cnn_bgru_sa.png')  # Save the plot as a PNG file
plt.close()

# Save Accuracy plot
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('accuracy_plot_cnn_bgru_sa.png')  # Save the plot as a PNG file
plt.close()

# Save Precision plot (custom metric)
plt.figure()
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.legend()
plt.title("Precision")
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.savefig('precision_plot_cnn_bgru_sa.png')  # Save the plot as a PNG file
plt.close()

# Save Recall plot (custom metric)
plt.figure()
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.legend()
plt.title("Recall")
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.savefig('recall_plot_cnn_bgru_sa.png')  # Save the plot as a PNG file
plt.close()

# Save F1-Score plot (custom metric)
plt.figure()
plt.plot(history.history['f1_score'], label='Train F1 Score')
plt.plot(history.history['val_f1_score'], label='Validation F1 Score')
plt.legend()
plt.title("F1 Score")
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.savefig('f1_score_plot_cnn_bgru_sa.png')  # Save the plot as a PNG file
plt.close()


In [18]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, RocCurveDisplay

# ----------- Get predictions -----------
# Get true labels and predicted probabilities
test_set.reset()  # Important to reset the generator
Y_true = test_set.classes  # True labels
Y_pred_prob = model.predict(test_set)
Y_pred = (Y_pred_prob >= 0.5).astype(int).flatten()  # Convert probabilities to binary predictions

# ----------- Confusion Matrix -----------
cm = confusion_matrix(Y_true, Y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.savefig('confusion_matrix_cnn_bgru_sa.png')
plt.close()

# ----------- Classification Report (Precision, Recall, F1) -----------
report = classification_report(Y_true, Y_pred, target_names=list(test_set.class_indices.keys()))
print("Classification Report:\n", report)

# ----------- AUC - ROC Curve -----------
fpr, tpr, thresholds = roc_curve(Y_true, Y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.savefig('roc_auc_curve_cnn_bgru_sa.png')
plt.close()


100/100 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step
Classification Report:
                      precision    recall  f1-score   support

Monkeypox_augmented       0.42      0.39      0.40      1428
   Others_augmented       0.53      0.56      0.55      1764

           accuracy                           0.49      3192
          macro avg       0.48      0.48      0.48      3192
       weighted avg       0.48      0.49      0.48      3192



In [19]:
# Save the trained model
model.save("monkeypox_cnn_bgru_sa.h5")

In [20]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [42]:
img_path = "C:/Users/KIIT/Music/Monkeypox/Kaggle MP images/archive (2)/Augmented Images/Augmented Images/Monkeypox_augmented/M01_01_02.jpg"

In [49]:
# Load and preprocess the image
img = image.load_img(img_path, target_size=(64, 64))  # Resize to match model input size
img_array = image.img_to_array(img) / 255.0  # Normalize pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

In [50]:
# Make prediction
prediction = model.predict(img_array)
predicted_class = prediction[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


In [46]:
predicted_class

0.00091102475

In [48]:
img_path = "C:/Users/KIIT/Music/Monkeypox/Kaggle MP images/archive (2)/Augmented Images/Augmented Images/Others_augmented/NM01_01_00.jpg"

In [51]:
# Set a decision threshold
threshold = 0.5  # Adjust if needed based on model performance

# Output the result
if predicted_class < threshold:
    print(f"The image {img_path} is classified as Monkeypox Positive.")
else:
    print(f"The image {img_path} is classified as Monkeypox Negative.")

The image C:/Users/KIIT/Music/Monkeypox/Kaggle MP images/archive (2)/Augmented Images/Augmented Images/Others_augmented/NM01_01_00.jpg is classified as Monkeypox Negative.


In [38]:
model.save("monkeypox_cnn_bgru_sa.keras")